# Advent of Code 2024 Day 17 (Chronospatial Computer)

## Part 1

In [1]:
from dataclasses import dataclass
from enum import auto, Enum
import math

In [2]:
OperandType = Enum("OperandType", "COMBO LITERAL")

class Opcode(Enum):
    ADV = 0
    BXL = auto()
    BST = auto()
    JNZ = auto()
    BXC = auto()
    OUT = auto()
    BDV = auto()
    CDV = auto()

    @property
    def operand_type(self) -> OperandType | None:
        match self:
            case self.ADV | self.BST | self.OUT | self.BDV | self.CDV:
                operand_type = OperandType.COMBO
            case self.BXL | self.JNZ:
                operand_type = OperandType.LITERAL
            case _:
                operand_type = None

        return operand_type

@dataclass(slots=True, frozen=True)
class Operand:
    value: int
    category: OperandType

@dataclass(slots=True)
class Pointer:
    position: int
    is_on_opcode: bool = True

    def move(self):
        self.position += 1
        self.is_on_opcode = False if self.is_on_opcode else True

    def jump(self, new_pos):
        self.position = new_pos
        self.is_on_opcode = True

@dataclass(slots=True, frozen=True)
class Instruction:
    opcode: Opcode
    operand: Operand

    def operate(self, registers: dict):
        operand_val = self.get_operand_value(registers)
                        
        match self.opcode:
            case Opcode.ADV:
                registers["A"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.BDV:
                registers["B"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.CDV:
                registers["C"] = math.trunc(registers["A"] / (2**operand_val))
            case Opcode.BXL:
                registers["B"] = registers["B"] ^ operand_val
            case Opcode.BST:
                registers["B"] = operand_val % 8
            case Opcode.BXC:
                registers["B"] = registers["B"] ^ registers["C"]
            case _:
                result = None

        return registers

    def get_output(self, registers):
        output = self.get_operand_value(registers) % 8
        return output

    def get_operand_value(self, registers):
        match self.operand.category:
            case OperandType.COMBO:
                match self.operand.value:
                    case n if all([0 <= n, n <= 3]):
                        operand_result = self.operand.value
                    case 4:
                        operand_result = registers["A"]
                    case 5:
                        operand_result = registers["B"]
                    case 6:
                        operand_result = registers["C"]
                    case _:
                        raise ValueError("invalid operand value [0-6]")
            case OperandType.LITERAL:
                if any([self.operand.value < 0, self.operand.value > 6]):
                    raise ValueError("invalid operand value [0-6]")
                operand_result = self.operand.value
            case _:
                operand_result = None

        return operand_result


def program_is_finished(codes: list, pointer: Pointer):
    return pointer.position > len(codes) - 1

In [3]:
pointer = Pointer(0)
registers = dict()
output = []

with open("data/instructions.txt", "r") as file:
    content = [line.strip() for line in file.readlines()]
    
    for row in content:
        if "Register" in row:
            register_row = row.split()
            key = register_row[-2].replace(":", "")
            value = register_row[-1]
            registers[key] = int(value)
        elif "Program" in row:
            codes_list = row.split()
            insts = row.replace("Program: ", "").split(",")
            
            while not program_is_finished(insts, pointer):
                if pointer.is_on_opcode:
                    instruction = None
                    opcode = Opcode(int(insts[pointer.position]))
                else:
                    operand = Operand(int(insts[pointer.position]), opcode.operand_type)
                    instruction = Instruction(opcode, operand)

                    if instruction.opcode not in { Opcode.JNZ, Opcode.OUT }:
                        result = instruction.operate(registers)
                    elif instruction.opcode == Opcode.JNZ and registers["A"] != 0:
                        pointer.jump(instruction.operand.value)
                        continue
                    elif instruction.opcode == Opcode.OUT:
                        output.append(instruction.get_output(registers))
                        
                pointer.move()

                
output = ",".join([str(out) for out in output])
output

'6,7,5,2,1,3,5,1,7'

In [4]:
with open("answer.txt", "w") as file:
    file.writelines([output])